In [54]:
# Cargo todas las librerias necesarias para el modelo
import pandas as pd
import math
import time
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import mifs
import random
from lssvr import LSSVR
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [2]:
def create_dataset(dataset,look_back):
    k=np.shape(dataset)[0]
    dataset=dataset.reshape(1,k)
    datax=np.zeros((k-look_back-23,look_back))
    for i in range(0,look_back,1):
        datax[:,i]=dataset[:,i:(k-look_back+i-23)]
    dataY=np.zeros((k-look_back-23,24))
    for i in range(0,24,1):
        dataY[:,i]=dataset[:,(look_back+i):(k-23+i)]
    return datax,dataY

In [3]:
class PSO:
    def __init__(self, parameters,T0=3,Tg=5,Tgz=10,w=0.8,c1=2,c2=2):
        # 初始化
        self.NGEN = parameters[0]    # 迭代的代数
        self.pop_size = parameters[1]    # 种群大小
        self.var_num = len(parameters[2])     # 变量个数
        self.bound = []                 # 变量的约束范围
        self.bound.append(parameters[2])
        self.bound.append(parameters[3])
        self.cp, self.cg = c1, c2
        self.w = w
        self.T0=T0*np.ones((self.pop_size,1))
        self.Tg=Tg
        self.pop_x = np.zeros((self.pop_size, self.var_num))    # 所有粒子的位置
        #self.pop_v = np.zeros((self.pop_size, self.var_num))    # 所有粒子的速度
        self.p_best = np.zeros((self.pop_size, self.var_num))   # 每个粒子最优的位置
        self.g_best = np.zeros((1, self.var_num))   # 全局最优的位置
        self.tg=0
        self.t0= np.zeros((self.pop_size,1))
        # 初始化第0代初始全局最优解
        temp = -1
        for i in range(self.pop_size):
            for j in range(self.var_num):
                self.pop_x[i][j] = random.uniform(self.bound[0][j], self.bound[1][j])
                #self.pop_v[i][j] = random.uniform(0, 1)
            self.p_best[i] = self.pop_x[i]      # 储存最优的个体
            fit = self.fitness(self.p_best[i])
            if fit > temp:
                self.g_best = self.p_best[i]
                temp = fit
    def fitness(self, ind_var):
        X = trainx
        y = trainy
        x1 = ind_var[0]
        x2 = ind_var[1]
        if x1==0:x1=0.001
        if x2==0:x2=0.001
        clf = LSSVR(kernel='rbf',C=x1,gamma=x2)
        clf.fit(X, y)
        predictval=clf.predict(validx)
       # print("R2 = ",metrics.r2_score(validyimf1[:,0],predictval)) # R2
        return  metrics.r2_score(validy,predictval)
    
    def update_operator(self, pop_size):
        """
        更新算子：更新下一时刻的位置和速度
        """
        r3= np.where(self.t0 >self.T0, \
            np.random.uniform(low=0,high=1,size=(self.pop_size, self.var_num)),1)
        r4= np.where(self.tg >self.Tg,\
            np.random.uniform(low=0,high=1,size=(1,self.var_num)),1)
        r1 = np.random.rand(self.pop_size, self.var_num)
        r2 = np.random.rand(self.pop_size, self.var_num)
        self.pop_x=self.w *self.pop_x+self.cp*r1*(r3*self.p_best-self.pop_x)+\
               self.cg*r2*(r4*self.g_best - self.pop_x)
        for i in range(pop_size):
            # 越界保护
            for j in range(self.var_num):
                if self.pop_x[i][j] < self.bound[0][j]:
                    self.pop_x[i][j] = self.bound[0][j]
                if self.pop_x[i][j] > self.bound[1][j]:
                    self.pop_x[i][j] = self.bound[1][j]
            # 更新p_best和g_best
            if self.fitness(self.pop_x[i]) > self.fitness(self.p_best[i]):
                self.p_best[i] = self.pop_x[i]
            if self.fitness(self.pop_x[i]) > self.fitness(self.g_best):
                self.g_best = self.pop_x[i]
    def main(self):
        popobj = []
        self.ng_best = np.zeros((1, self.var_num))[0]
        for gen in range(self.NGEN):
            self.update_operator(self.pop_size)
            popobj.append(self.fitness(self.g_best))
            if self.fitness(self.g_best) > self.fitness(self.ng_best):
                self.ng_best = self.g_best.copy()
        print('最好的位置：{}'.format(self.ng_best))
        print('最大的函数值：{}'.format(self.fitness(self.ng_best)))

In [31]:
NGEN = 20
popsize = 20
low = [0.001,0.001]
up = [10,100]
parameters = [NGEN, popsize, low, up]

In [5]:
imf1 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf1.csv").values
imf2 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf2.csv").values
imf3 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf3.csv").values
imf4 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf4.csv").values
imf5 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf5.csv").values
imf6 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf6.csv").values
look_back=168
trainximf1,trainyimf1=create_dataset(imf1[0:672],look_back)
validximf1,validyimf1=create_dataset(imf1[480:912],look_back)
testximf1,testyimf1=create_dataset(imf1[720:1009],look_back)#12月1号
trainximf2,trainyimf2=create_dataset(imf2[0:672],look_back)
validximf2,validyimf2=create_dataset(imf2[480:912],look_back)
testximf2,testyimf2=create_dataset(imf2[720:1009],look_back)#12月1号

trainximf3,trainyimf3=create_dataset(imf3[0:672],look_back)
validximf3,validyimf3=create_dataset(imf3[480:912],look_back)
testximf3,testyimf3=create_dataset(imf3[720:1009],look_back)#12月1号

trainximf4,trainyimf4=create_dataset(imf4[0:672],look_back)
validximf4,validyimf4=create_dataset(imf4[480:912],look_back)
testximf4,testyimf4=create_dataset(imf4[720:1009],look_back)#12月1号

trainximf5,trainyimf5=create_dataset(imf5[0:672],look_back)
validximf5,validyimf5=create_dataset(imf5[480:912],look_back)
testximf5,testyimf5=create_dataset(imf5[720:1009],look_back)#12月1号

trainximf6,trainyimf6=create_dataset(imf6[0:672],look_back)
validximf6,validyimf6=create_dataset(imf6[480:912],look_back)
testximf6,testyimf6=create_dataset(imf6[720:1009],look_back)#12月1号

feat_selector = mifs.MutualInformationFeatureSelector(method='MRMR',n_features=8,categorical=False)
s=feat_selector.fit(trainximf1,trainyimf1[:,0])
mrmrimf1=list(set(s.ranking_))
s=feat_selector.fit(trainximf2,trainyimf2[:,0])
mrmrimf2=list(set(s.ranking_))
s=feat_selector.fit(trainximf3,trainyimf3[:,0])
mrmrimf3=list(set(s.ranking_))
s=feat_selector.fit(trainximf4,trainyimf4[:,0])
mrmrimf4=list(set(s.ranking_))
s=feat_selector.fit(trainximf5,trainyimf5[:,0])
mrmrimf5=list(set(s.ranking_))
s=feat_selector.fit(trainximf6,trainyimf6[:,0])
mrmrimf6=list(set(s.ranking_))

In [51]:
for j in range(2,9,1):
    trainx=trainximf1[:,mrmrimf1]
    trainy=trainyimf1[:,0]
    validx=validximf1[:,mrmrimf1]
    validy=validyimf1[:,0]
    testx=testximf1[:,mrmrimf1]
    testy=testyimf1[:,0]
    pso = PSO(parameters,T0=3,Tg=5,Tgz=10,w=0.8,c1=2,c2=2)
    pso.main() 
    model = LSSVR(kernel='rbf',C=pso.ng_best[0],gamma=pso.ng_best[1])
    model.fit(trainx, trainy)
    predy=np.zeros((len(testyimf1),0))
    testxx=testximf1
    for i in range(0,24,1):
        y_hat = model.predict(testx)
        predy=np.c_[predy,y_hat]
        testxx=np.c_[testxx,y_hat]
        testx=testxx[:,list(map(lambda x:x+i+1,mrmrimf1))]
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/IMF1d"+str(j)+".csv"
    dt.to_csv(url,index=0)
    print(j)

最好的位置：[10.          0.39720208]
最大的函数值：0.9886762472384985
2
最好的位置：[10.          0.41688447]
最大的函数值：0.9886583342323267
3
最好的位置：[10.          0.38574759]
最大的函数值：0.9886683014494655
4
最好的位置：[10.          0.40439028]
最大的函数值：0.988674167189949
5
最好的位置：[10.          0.40971165]
最大的函数值：0.9886692782134703
6
最好的位置：[10.         0.3921608]
最大的函数值：0.988674499498374
7
最好的位置：[10.          0.40242456]
最大的函数值：0.9886752537417456
8


In [52]:
def dgSVR(trainx,trainy,validx,validy,testx,testy,testximf1,mrmrimf1):
    pso = PSO(parameters,T0=3,Tg=5,Tgz=10,w=0.8,c1=2,c2=2)
    pso.main() 
    model = LSSVR(kernel='rbf',C=pso.ng_best[0],gamma=pso.ng_best[1])
    model.fit(trainx, trainy)
    predy=np.zeros((len(testyimf1),0))
    testxx=testximf1
    for i in range(0,24,1):
        y_hat = model.predict(testx)
        predy=np.c_[predy,y_hat]
        testxx=np.c_[testxx,y_hat]
        testx=testxx[:,list(map(lambda x:x+i+1,mrmrimf1))]
    return predy

In [53]:
for j in range(1,9,1):
    trainx=trainximf2[:,mrmrimf2]
    trainy=trainyimf2[:,0]
    validx=validximf2[:,mrmrimf2]
    validy=validyimf2[:,0]
    testx=testximf2[:,mrmrimf2]
    testy=testyimf2[:,0]
    predy=dgSVR(trainx,trainy,validx,validy,testx,testy,testximf2,mrmrimf2)
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/IMF2d"+str(j)+".csv"
    dt.to_csv(url,index=0)
    
    trainx=trainximf3[:,mrmrimf3]
    trainy=trainyimf3[:,0]
    validx=validximf3[:,mrmrimf3]
    validy=validyimf3[:,0]
    testx=testximf3[:,mrmrimf3]
    testy=testyimf3[:,0]
    predy=dgSVR(trainx,trainy,validx,validy,testx,testy,testximf3,mrmrimf3)
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/IMF3d"+str(j)+".csv"
    dt.to_csv(url,index=0)
    
    trainx=trainximf4[:,mrmrimf4]
    trainy=trainyimf4[:,0]
    validx=validximf4[:,mrmrimf4]
    validy=validyimf4[:,0]
    testx=testximf4[:,mrmrimf4]
    testy=testyimf4[:,0]
    predy=dgSVR(trainx,trainy,validx,validy,testx,testy,testximf4,mrmrimf4)
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/IMF4d"+str(j)+".csv"
    dt.to_csv(url,index=0)
    
    trainx=trainximf5[:,mrmrimf5]
    trainy=trainyimf5[:,0]
    validx=validximf5[:,mrmrimf5]
    validy=validyimf5[:,0]
    testx=testximf5[:,mrmrimf5]
    testy=testyimf5[:,0]
    predy=dgSVR(trainx,trainy,validx,validy,testx,testy,testximf5,mrmrimf5)
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/IMF5d"+str(j)+".csv"
    dt.to_csv(url,index=0)
    
    trainx=trainximf6[:,mrmrimf6]
    trainy=trainyimf6[:,0]
    validx=validximf6[:,mrmrimf6]
    validy=validyimf6[:,0]
    testx=testximf6[:,mrmrimf6]
    testy=testyimf6[:,0]
    predy=dgSVR(trainx,trainy,validx,validy,testx,testy,testximf6,mrmrimf6)
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/I MF6d"+str(j)+".csv"
    dt.to_csv(url,index=0)
    print(j)

最好的位置：[10.          2.73549033]
最大的函数值：0.9975629320431241
最好的位置：[10.          2.80795614]
最大的函数值：0.9729354326686588
最好的位置：[ 10. 100.]
最大的函数值：0.9316689517760509
最好的位置：[10.         77.89835104]
最大的函数值：0.9606844572484372
最好的位置：[10.        31.2490275]
最大的函数值：0.8017962414609835
1
最好的位置：[10.          2.74406252]
最大的函数值：0.9975628043285555
最好的位置：[10.          2.75205136]
最大的函数值：0.9729219099989115
最好的位置：[ 10. 100.]
最大的函数值：0.9316689517760509
最好的位置：[10.         77.72545007]
最大的函数值：0.9606848113960956
最好的位置：[10.         28.99840176]
最大的函数值：0.8018214108130071
2
最好的位置：[10.          2.64005185]
最大的函数值：0.9975595190419353
最好的位置：[10.          2.68460855]
最大的函数值：0.972922613942115
最好的位置：[ 10. 100.]
最大的函数值：0.9316689517760509
最好的位置：[10.         77.83023164]
最大的函数值：0.9606843785782477
最好的位置：[10.         29.68045779]
最大的函数值：0.8017985010545302
3
最好的位置：[10.          2.71306669]
最大的函数值：0.9975631359028676
最好的位置：[10.          2.71888625]
最大的函数值：0.9729285853262108
最好的位置：[ 10. 100.]
最大的函数值：0.9316689517760509
最好的位置：[10